# BÀI THỰC HÀNH 4: MẠNG NEURAL HỒI QUY

<b>Hướng dẫn nộp bài:</b> Các bạn commit và push code lên github, sử dụng file txt đặt tên theo cú pháp <MSSV>.txt chứa đường link dẫn đến github của bài thực hành và nộp file txt này tên courses.

Bộ dữ liệu sử dụng: [PhoMT](https://drive.google.com/drive/folders/1ksAAeUq2b4u_hiKpyzaEDNzfQDHoB1cI?usp=sharing).

#### Bài 1: Xây dựng kiến trúc Encoder-Decoder gồm 5 lớp LSTM cho module encoder và 5 lớp LSTM cho module decoder, với hidden size là 256, cho bài toán dịch máy từ tiếng Anh sang tiếng Việt. Huấn luyện mô hình này trên bộ dữ liệu PhoMT sử dụng Adam làm phương thức tối ưu tham số. Đánh giá độ hiệu quả của mô hình sử dụng các độ đo BLEU (BLEU@1, BLEU@2, BLEU@3, BLEU@4), ROUGE, và Meteor.

In [ ]:
# Cài đặt gdown (nếu chưa có)
!pip install gdown --upgrade

import gdown

folder_id = "186OAOuSEYEDVcry7WP5UBdqECXo26QAb"
gdown.download_folder(id=folder_id, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1hoTd2hFwjSeFThlPm6YpN0NW5ePXS3Jc small-dev.json
Processing file 1_3L25SH1_jaEfOjpmpgnfMik4N3MxSyn small-test.json
Processing file 1-eG6FeF-v__rsf77iWurddahXbyjTYh5 small-train.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1hoTd2hFwjSeFThlPm6YpN0NW5ePXS3Jc
To: /content/small-PhoMT/small-dev.json
100%|██████████| 594k/594k [00:00<00:00, 123MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_3L25SH1_jaEfOjpmpgnfMik4N3MxSyn
To: /content/small-PhoMT/small-test.json
100%|██████████| 669k/669k [00:00<00:00, 121MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-eG6FeF-v__rsf77iWurddahXbyjTYh5
To: /content/small-PhoMT/small-train.json
100%|██████████| 5.68M/5.68M [00:00<00:00, 309MB/s]
Download completed


['/content/small-PhoMT/small-dev.json',
 '/content/small-PhoMT/small-test.json',
 '/content/small-PhoMT/small-train.json']

In [ ]:
import json

def load_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Giả sử định dạng là list các dict với key "en" và "vi"
    pairs = [(item["english"], item["vietnamese"]) for item in data]
    return pairs

train_pairs = load_data("/content/small-PhoMT/small-train.json")
dev_pairs = load_data("/content/small-PhoMT/small-dev.json")
test_pairs = load_data("/content/small-PhoMT/small-test.json")

print(f"Train: {len(train_pairs)} câu")
print(f"Dev: {len(dev_pairs)} câu")
print(f"Test: {len(test_pairs)} câu")

Train: 20000 câu
Dev: 2000 câu
Test: 2000 câu


In [28]:
import re
import unicodedata
from collections import Counter

def normalize_text(text):
    # Chuẩn hóa unicode và chuyển về lowercase (chỉ với tiếng Anh)
    text = unicodedata.normalize('NFC', text)
    text = text.lower()
    # Loại bỏ ký tự đặc biệt, giữ lại chữ cái, số và khoảng trắng
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Tách từ đơn giản (word level)
def tokenize_en(text):
    return normalize_text(text).split()

def tokenize_vi(text):
    # Tiếng Việt giữ nguyên dấu, chỉ chuẩn hóa unicode và loại bỏ ký tự lạ
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'[^a-zA-Z0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠ-ỹ\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.split()

# Xây dựng từ vựng
class Vocabulary:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        self.add_word('<PAD>')
        self.add_word('<SOS>')
        self.add_word('<EOS>')
        self.add_word('<UNK>')

    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def add_sentence(self, sentence, tokenizer):
        for word in tokenizer(sentence):
            self.add_word(word)

    def __len__(self):
        return self.idx

# Xây dựng vocab cho source (en) và target (vi)
def build_vocabs(train_pairs, max_vocab_size=8000):
    src_vocab = Vocabulary()
    tgt_vocab = Vocabulary()

    src_counter = Counter()
    tgt_counter = Counter()

    for en, vi in train_pairs:
        src_counter.update(tokenize_en(en))
        tgt_counter.update(tokenize_vi(vi))

    # Chỉ lấy các từ xuất hiện ít nhất 2 lần + top frequent
    for word, freq in src_counter.items():
        if freq >= 2:
            src_vocab.add_word(word)
    for word, freq in tgt_counter.items():
        if freq >= 2:
            tgt_vocab.add_word(word)

    # Giới hạn cứng vocab size
    if len(src_vocab) > max_vocab_size:
        # giữ lại top (max_vocab_size - 4) từ phổ biến nhất
        top_words = [w for w, _ in src_counter.most_common(max_vocab_size - 4)]
        src_vocab = Vocabulary()
        for w in top_words:
            src_vocab.add_word(w)

    if len(tgt_vocab) > max_vocab_size:
        top_words = [w for w, _ in tgt_counter.most_common(max_vocab_size - 4)]
        tgt_vocab = Vocabulary()
        for w in top_words:
            tgt_vocab.add_word(w)

    return src_vocab, tgt_vocab

src_vocab, tgt_vocab = build_vocabs(train_pairs, max_vocab_size=6000)

print(f"Vocab EN mới: {len(src_vocab)} từ")
print(f"Vocab VI mới: {len(tgt_vocab)} từ")

Vocab EN mới: 6000 từ
Vocab VI mới: 5000 từ


In [29]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class TranslationDataset(Dataset):
    def __init__(self, pairs, src_vocab, tgt_vocab):
        self.pairs = pairs
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        en, vi = self.pairs[idx]

        src = [self.src_vocab.word2idx.get(w, self.src_vocab.word2idx['<UNK>']) for w in tokenize_en(en)]
        src = [self.src_vocab.word2idx['<SOS>']] + src + [self.src_vocab.word2idx['<EOS>']]

        tgt = [self.tgt_vocab.word2idx.get(w, self.tgt_vocab.word2idx['<UNK>']) for w in tokenize_vi(vi)]
        tgt = [self.tgt_vocab.word2idx['<SOS>']] + tgt + [self.tgt_vocab.word2idx['<EOS>']]

        return torch.tensor(src), torch.tensor(tgt)

def collate_fn(batch):
    srcs, tgts = zip(*batch)
    srcs_padded = pad_sequence(srcs, batch_first=True, padding_value=src_vocab.word2idx['<PAD>'])
    tgts_padded = pad_sequence(tgts, batch_first=True, padding_value=tgt_vocab.word2idx['<PAD>'])
    return srcs_padded, tgts_padded

batch_size = 64

train_dataset = TranslationDataset(train_pairs, src_vocab, tgt_vocab)
dev_dataset   = TranslationDataset(dev_pairs, src_vocab, tgt_vocab)
test_dataset  = TranslationDataset(test_pairs, src_vocab, tgt_vocab)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dev_loader   = DataLoader(dev_dataset,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


print("DataLoader đã sẵn sàng!")

DataLoader đã sẵn sàng!


In [ ]:
import torch.nn as nn
import random

class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=5, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=src_vocab.word2idx['<PAD>'])
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        emb = self.embedding(x)
        _, (hidden, cell) = self.lstm(emb)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=5, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=tgt_vocab.word2idx['<PAD>'])
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        emb = self.embedding(x)
        output, (hidden, cell) = self.lstm(emb, (hidden, cell))
        pred = self.fc(output)
        return pred, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        batch_size = src.size(0)
        tgt_len = tgt.size(1)
        tgt_vocab_size = len(tgt_vocab)

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(src.device)

        hidden, cell = self.encoder(src)

        input_token = tgt[:, 0].unsqueeze(1)  # <SOS>

        for t in range(1, tgt_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t, :] = output.squeeze(1)

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(-1)
            input_token = tgt[:, t].unsqueeze(1) if teacher_force else top1

        return outputs

# Khởi tạo mô hình
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embed_size = 256
hidden_size = 256
num_layers = 5

encoder = Encoder(len(src_vocab), embed_size, hidden_size, num_layers)
decoder = Decoder(len(tgt_vocab), embed_size, hidden_size, num_layers)
model = Seq2Seq(encoder, decoder).to(device)



print(model)
print(f"Số tham số: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6000, 256, padding_idx=0)
    (lstm): LSTM(256, 256, num_layers=5, batch_first=True, dropout=0.3)
  )
  (decoder): Decoder(
    (embedding): Embedding(5000, 256, padding_idx=0)
    (lstm): LSTM(256, 256, num_layers=5, batch_first=True, dropout=0.3)
    (fc): Linear(in_features=256, out_features=5000, bias=True)
  )
)
Số tham số: 9,364,360


In [31]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=tgt_vocab.word2idx['<PAD>'])

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for src, tgt in loader:
        src, tgt = src.to(device), tgt.to(device)

        optimizer.zero_grad()
        output = model(src, tgt, teacher_forcing_ratio=0.5)

        loss = criterion(output[:, 1:].reshape(-1, output.size(-1)), tgt[:, 1:].flatten())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

# Huấn luyện
epochs = 30

for epoch in range(1, epochs + 1):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    print(f"Epoch {epoch:2d} | Loss: {loss:.4f}")

Epoch  1 | Loss: 6.4107
Epoch  2 | Loss: 6.2936
Epoch  3 | Loss: 6.2811
Epoch  4 | Loss: 6.2665
Epoch  5 | Loss: 6.2289
Epoch  6 | Loss: 6.1695
Epoch  7 | Loss: 6.0858
Epoch  8 | Loss: 5.9998
Epoch  9 | Loss: 5.9300
Epoch 10 | Loss: 5.8623
Epoch 11 | Loss: 5.7979
Epoch 12 | Loss: 5.7451
Epoch 13 | Loss: 5.6796
Epoch 14 | Loss: 5.6189
Epoch 15 | Loss: 5.5692
Epoch 16 | Loss: 5.5186
Epoch 17 | Loss: 5.4794
Epoch 18 | Loss: 5.4349
Epoch 19 | Loss: 5.3976
Epoch 20 | Loss: 5.3670
Epoch 21 | Loss: 5.3219
Epoch 22 | Loss: 5.2946
Epoch 23 | Loss: 5.2617
Epoch 24 | Loss: 5.2360
Epoch 25 | Loss: 5.2141
Epoch 26 | Loss: 5.1783
Epoch 27 | Loss: 5.1506
Epoch 28 | Loss: 5.1240
Epoch 29 | Loss: 5.1140
Epoch 30 | Loss: 5.0949


In [32]:
!pip install rouge_score

In [33]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk import word_tokenize
from rouge_score import rouge_scorer

# Hàm decode câu từ list indices → chuỗi text
def decode_sentence(indices, vocab):
    words = []
    for idx in indices:
        if idx == vocab.word2idx['<EOS>']:
            break
        if idx not in [vocab.word2idx['<PAD>'], vocab.word2idx['<SOS>'], vocab.word2idx['<UNK>']]:
            word = vocab.idx2word.get(idx, '<UNK>')
            words.append(word)
    return ' '.join(words)

# Hàm đánh giá
def evaluate(model, loader):
    model.eval()
    hypotheses = []  # dự đoán
    references = []  # ground truth

    with torch.no_grad():
        for src, tgt in loader:
            src = src.to(device)

            # Encode source
            hidden, cell = model.encoder(src)

            batch_size = src.size(0)
            max_len = 50  # giới hạn độ dài sinh

            # Bắt đầu decoding với <SOS>
            input_token = torch.full((batch_size, 1), tgt_vocab.word2idx['<SOS>'],
                                   dtype=torch.long, device=device)
            pred_sequences = []

            for _ in range(max_len):
                output, hidden, cell = model.decoder(input_token, hidden, cell)
                next_token = output.argmax(dim=-1)  # greedy
                input_token = next_token
                pred_sequences.append(next_token.squeeze(1).cpu())

            # Ghép lại thành tensor (batch, seq_len)
            pred_seq = torch.stack(pred_sequences, dim=1)

            # Decode từng câu trong batch
            for i in range(batch_size):
                pred_str = decode_sentence(pred_seq[i], tgt_vocab)
                ref_str  = decode_sentence(tgt[i].tolist(), tgt_vocab)

                hypotheses.append(pred_str)
                references.append(ref_str)

    print(f"Đã sinh {len(hypotheses)} câu dự đoán.\n")

    smoothie = SmoothingFunction().method4

    bleu1_scores = [sentence_bleu([ref.split()], hyp.split(), weights=(1, 0, 0, 0), smoothing_function=smoothie)
                    for hyp, ref in zip(hypotheses, references)]
    bleu2_scores = [sentence_bleu([ref.split()], hyp.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
                    for hyp, ref in zip(hypotheses, references)]
    bleu3_scores = [sentence_bleu([ref.split()], hyp.split(), weights=(1/3, 1/3, 1/3, 0), smoothing_function=smoothie)
                    for hyp, ref in zip(hypotheses, references)]
    bleu4_scores = [sentence_bleu([ref.split()], hyp.split(), weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
                    for hyp, ref in zip(hypotheses, references)]

    print(f"BLEU@1: {np.mean(bleu1_scores):.4f}")
    print(f"BLEU@2: {np.mean(bleu2_scores):.4f}")
    print(f"BLEU@3: {np.mean(bleu3_scores):.4f}")
    print(f"BLEU@4: {np.mean(bleu4_scores):.4f}")

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougeL = [], [], []


gdown.download_folder(id=folder_id, quiet=False, use_cookies=False)
    for hyp, ref in zip(hypotheses, references):
        scores = scorer.score(ref, hyp)
        rouge1.append(scores['rouge1'].fmeasure)
        rouge2.append(scores['rouge2'].fmeasure)
        rougeL.append(scores['rougeL'].fmeasure)

    print(f"ROUGE-1: {np.mean(rouge1):.4f}")
    print(f"ROUGE-2: {np.mean(rouge2):.4f}")
    print(f"ROUGE-L: {np.mean(rougeL):.4f}")

    meteor_scores = []
    for hyp, ref in zip(hypotheses, references):
        # Tokenize trước khi tính METEOR
        hyp_tokens = word_tokenize(hyp)
        ref_tokens = word_tokenize(ref)
        score = meteor_score([ref_tokens], hyp_tokens)  # list của reference
        meteor_scores.append(score)



    print(f"METEOR: {np.mean(meteor_scores):.4f}")

print("Đang đánh giá trên tập test...")
evaluate(model, test_loader)

Đang đánh giá trên tập test...
Đã sinh 2000 câu dự đoán.

BLEU@1: 0.0000
BLEU@2: 0.0000
BLEU@3: 0.0000
BLEU@4: 0.0000
ROUGE-1: 0.0000
ROUGE-2: 0.0000
ROUGE-L: 0.0000
METEOR: 0.0000
